In [1]:
#Importation
import pickle
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# On ramene les données du prétraitement du dataframe
df = pd.read_pickle("../PICKLE/dataframe_labelise.pkl")
df

,No DESIGNATION,COMMENTAIRE DE SIG,COMMENTAIRE,BLOC NOTE,COMMLITIGEPIDI,Reussite
0,299568158,NaN,# Obtenu MARCHAND CHRISTOPHE le 26/04/2023 a 1...,NaN,\,False
1,297892193,NaN,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\\$GM$ Un rdv a ete repris par le technicien p...,True
2,297897767,NaN,/DMS/#PTO_non existante#/Mohand ok raccordemen...,NaN,\\$GM$ CP Obtenu cliente le 13/04 Le client a...,True
3,223370510,NaN,/DMS/#PTO_non existante#/Installation complete...,NaN,\,True
4,299556603,NaN,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\,True
...,...,...,...,...,...,...
4183,223275005,NaN,NaN,Merci de cleturer le rendez vous du client car...,\\$GM$ Un rdv a ete repris par le tehcnicien p...,True
4184,296611510,NaN,/DMS/#PTO_existante#-#Malfacon PLP non detecte...,NaN,\,True
4185,223309738,NaN,/DMS/#PTO_non existante#/Raccordement effectue...,NaN,\,True
4186,298170718,NaN,/DMS/#PTO_non existante#/Raccordement en chamb...,NaN,\\$GM$ Obtenu M. Couchevellou au 0658359425 le...,True


## 3 - Nettoyage des textes
Cette partie a pour but de mettre au propre les commentaires pour avoir un dataframe propre.
Tout d'abord on a codé plusieurs fonctions pour enlever la ponctuation, enlever les codes de relève, mettre en minscule, modifier les abbréviation et les fautes de frappes communes...

En suite j'ai tout récupérer et mis dans un data frame avec uniquement le numéro d'identification, la réussite de l'opération et la liste des commentaires.

In [2]:
#Fonction pour enlever les code de relève des textes.
labels = ["rrc","dms","pad","tvc","etu","rmc","rmf","ann","ort","anc","pbc","reo"]
def enleve_releve_liste(commentaires):
    commentaires_propres = []
    for commentaire in commentaires :
        for label in labels :
            commentaire = commentaire.replace(label,'')
        commentaires_propres.append(commentaire)
    
    return commentaires_propres

In [3]:
#Fonction pour enlever les string vides d'une liste de strings
def supprimer_string_vide(liste):
    liste2 = []
    for i in liste :
        if i != "":
            liste2.append(i)
    return liste2

In [4]:
#Fonction pour remplacer les acronymes des techniciens

#Créons un dictionnaire
dico_mots = {"=":"",".":"","_":" ","-":" ","/":" ","$":"","#":"","teste":"test","nimporte":"n'importe","cable":"câble","c ble":"câble","tranch e":"tranchée","tranchee":"tranchée","tranche":"tranchée","tiquetage":"étiquetage","malfa":"malfaçon","pb":"point de branchement","mevo":"message vocal","pto":"point de terminaison optique","pto_non":"point de terminaison optique non","ligible":"éligible","racc":"raccordement","fa ade":"facade","propi taire":"propriétaire","detect e":"détectée"}
dico_mots

{'=': '',
 '.': '',
 '_': ' ',
 '-': ' ',
 '/': ' ',
 '$': '',
 '#': '',
 'teste': 'test',
 'nimporte': "n'importe",
 'cable': 'câble',
 'c ble': 'câble',
 'tranch e': 'tranchée',
 'tranchee': 'tranchée',
 'tranche': 'tranchée',
 'tiquetage': 'étiquetage',
 'malfa': 'malfaçon',
 'pb': 'point de branchement',
 'mevo': 'message vocal',
 'pto': 'point de terminaison optique',
 'pto_non': 'point de terminaison optique non',
 'ligible': 'éligible',
 'racc': 'raccordement',
 'fa ade': 'facade',
 'propi taire': 'propriétaire',
 'detect e': 'détectée'}

In [5]:
#Fonction pour appliquer les changements du dictionnaire
def anti_acronymes_grammar(liste):
    liste_retour = []
    for e in liste:
        for key,value in dico_mots.items():
            e = e.replace(key,value)
        liste_retour.append(e)
    
    return liste_retour

In [6]:
def lower_case(liste):
    return [e.lower() for e in liste]

In [7]:
def sans_numero_ponctuation(liste):
    liste_rendu = []
    pattern = r'\d+'
    for text in liste :
        text = re.sub('\[.*?\]', '', text)
        text = re.sub('\w*\d\w*', '', text)
        text = re.sub(pattern,'',text)
        text = re.sub('[‘’“”…]', '', text)
        text = re.sub('\n', '', text)
        liste_rendu.append(text)
    return liste_rendu

liste = ["kooz","90 FE","9"]
sans_numero_ponctuation(liste)

['kooz', ' FE', '']

In [8]:
#La fonction regroupant tous les nettoyages
def liste_propre(liste):
    #On met tout en minuscule
    liste = lower_case(liste)
    #On enleve les codes de releve
    liste = enleve_releve_liste(liste)
    #On enleve les signes et on remplaces les accronymes et erreurs principales de frappe
    liste = anti_acronymes_grammar(liste)
    #On enleve les nombres et la ponctuation
    liste = sans_numero_ponctuation(liste)
    return liste

In [9]:
#Récupération de tous les commentaires dans une liste de commentaires
def transformation_colonnes_commentaires(dataframe,colonnes,separateur):
    #Création de la liste
    colonne_commentaires = []
    
    #On va procéder ligne à ligne
    for i in dataframe.index :
        #On crée la liste des commentaires pour la ligne i
        commentaires_i = []
        
        #Pour chaque colonne on split le commentaire et on l'ajoute à la liste des commentaires de la ligne i.
        for colonne in colonnes :
            #On récupère le commentaire
            texte = dataframe.at[i,colonne]
            #S'il y en a un, on applique le séparateur et on rend ça propre
            if isinstance(texte, str) :
                commentaires_i = commentaires_i + liste_propre(texte.split(separateur))
         
        colonne_commentaires.append(commentaires_i)
        
    return colonne_commentaires

In [10]:
print(transformation_colonnes_commentaires(df,["COMMENTAIRE DE SIG","COMMENTAIRE","BLOC NOTE","COMMLITIGEPIDI"],'\\'))

[[' obtenu marchand christophe le    a  le client repe son rdv car n a pas encore termine les   il va contacter orange pour fixer un nouveau rdv ', '', ''], ['  point de terminaison optique existante malfaçon on plp non détectée  point de terminaison optique existante brassage au niveau du pm ont ok dae ok ', '', '', 'gm un rdv a ete repris par le technicien pour le   a partir de  apsp ', ''], ['  point de terminaison optique non existante mohand ok raccordementordement pavillon en aerien ok point de terminaison optique pose au salon a la demande du client val pm  val point de terminaison optique  delc ok  dae ok ', '', '', "gm cp obtenu cliente le    le client a bien compris les travaux a realiser et nous recontactera pour reprendre rdv des travaux finisapsm poser une gaine entre l'arrivee interieure (pavillon) et l'emplacement souhaite de la prise fibre", 'gm obtenu clt le   au  repris rdv le   a partir de  apil', "gm cp obtenu cliente le    le client a bien compris les travaux a rea

In [11]:
df["COMMENTAIRES"] = transformation_colonnes_commentaires(df,["COMMENTAIRE DE SIG","COMMENTAIRE","BLOC NOTE","COMMLITIGEPIDI"],'\\')
df = df.drop(columns=["COMMENTAIRE DE SIG","COMMENTAIRE","BLOC NOTE","COMMLITIGEPIDI"])

In [12]:
def drop_pas_comm(dataframe):
    for index in dataframe.index:
        if dataframe.loc[index,"COMMENTAIRES"] == []:
            dataframe = dataframe.drop(dataframe.index[index])
    return dataframe
df = drop_pas_comm(df)

In [13]:
df

,No DESIGNATION,Reussite,COMMENTAIRES
0,299568158,False,[ obtenu marchand christophe le a le clien...
1,297892193,True,[ point de terminaison optique existante malf...
2,297897767,True,[ point de terminaison optique non existante ...
3,223370510,True,[ point de terminaison optique non existante ...
4,299556603,True,[ point de terminaison optique existante malf...
...,...,...,...
4183,223275005,True,[merci de clrer le rendez vous du client car i...
4184,296611510,True,[ point de terminaison optique existante malf...
4185,223309738,True,[ point de terminaison optique non existante ...
4186,298170718,True,[ point de terminaison optique non existante ...


In [14]:
df.to_pickle("../PICKLE/df_liste_commentaires.pkl")